# Todo?

### Handle extreme data

remove meanwdspd outliers?

In [ ]:
# Call describe on df and transpose it due to the large number of columns
spread = df.describe().T

# precalculate interquartile range for ease of use in next calculation
IQR = spread['75%'] - spread['25%']

# create an outliers column which is either 3 IQRs below the first quartile or
# 3 IQRs above the third quartile
spread['outliers'] = (spread['min']<(spread['25%']-(3*IQR)))|(spread['max'] > (spread['75%']+3*IQR))

# just display the features containing extreme outliers
spread[spread.outliers]
# spread

### lasso regression and ridge regression to handle too many features?

lassso: seletive on features

ridge: limit features

$\uparrow$ multiple models and compare?

### Similarity-based learning and predict weather type?

need to re-do dataPreparation to include 'special' or 'important'

### Research correlation and select which features to be included.

The aim of this model is to predict the mean temperature with given historical data. By the nature of the linear regression model, the features used in the predictions have to have strong correlation with the target.

Hence, we need to study the correlation between the features and our target, mean temperature *meantemp*:

In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
df_nona.corr()[['meantemp']].sort_values('meantemp').T

/tmp/ipykernel_219/4203837516.py:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  df_nona.corr()[['meantemp']].sort_values('meantemp').T


,meanpres,meanpres_1,maxpres,maxpres_1,meanpres_2,maxpres_2,meanpres_3,maxpres_3,minpres,minpres_1,minpres_2,minpres_3,meanwdspd_1,meanwdspd_2,meanwdspd_3,meanwdspd,NW_wd,NW_wd_3,N_wd,N_wd_3,NW_wd_1,NW_wd_2,N_wd_2,N_wd_1,cloudy_2,cloudy_3,cloudy,cloudy_1,W_wd,W_wd_3,NE_wd,little_2,NE_wd_3,little_1,little,little_3,W_wd_2,W_wd_1,NE_wd_2,nocld_1,NE_wd_1,nocld_3,nocld,nocld_2,rain,some,rain_1,rain_2,much,SW_wd,rain_3,some_3,some_2,much_1,much_2,some_1,SW_wd_3,much_3,E_wd,SW_wd_2,E_wd_3,SW_wd_1,E_wd_2,E_wd_1,S_wd,S_wd_3,S_wd_2,S_wd_1,SE_wd,SE_wd_3,SE_wd_2,SE_wd_1,minhumi,minhumi_3,minhumi_2,maxhumi_3,minhumi_1,maxhumi_2,maxhumi,maxhumi_1,meanhumi_3,meanhumi,meanhumi_2,meanhumi_1,mindewpt_3,mindewpt_2,mindewpt,mindewpt_1,meandewpt_3,maxdewpt_3,maxdewpt_2,meandewpt_2,maxdewpt_1,meandewpt_1,meandewpt,maxdewpt,maxtemp_3,mintemp_3,maxtemp_2,mintemp_2,mintemp_1,maxtemp_1,meantemp_3,mintemp,meantemp_2,maxtemp,meantemp_1,meantemp
meantemp,-0.888291,-0.884047,-0.877442,-0.877294,-0.862123,-0.857301,-0.849329,-0.843851,-0.832205,-0.825793,-0.804286,-0.793083,-0.168622,-0.148217,-0.12873,-0.126531,-0.11651,-0.108752,-0.106826,-0.100569,-0.094822,-0.09394,-0.093232,-0.087293,-0.044946,-0.042317,-0.041403,-0.036049,-0.017963,-0.013365,-0.009491,-0.000559,0.000774,0.002434,0.003328,0.004337,0.006081,0.011069,0.014234,0.017199,0.019181,0.020419,0.024166,0.025272,0.028135,0.030129,0.031013,0.031659,0.033692,0.035432,0.035463,0.036888,0.038346,0.038692,0.038895,0.039471,0.042452,0.045934,0.053197,0.061853,0.069335,0.069406,0.083109,0.085941,0.117106,0.118317,0.135669,0.146929,0.151426,0.153418,0.166706,0.172652,0.287682,0.29533,0.303513,0.305239,0.312512,0.31455,0.326012,0.330528,0.335664,0.338495,0.349106,0.364659,0.826478,0.838962,0.851051,0.854844,0.879245,0.880397,0.889723,0.890736,0.906198,0.908301,0.911412,0.918413,0.92386,0.930355,0.934199,0.936123,0.948148,0.950347,0.958868,0.964997,0.966659,0.967442,0.981401,1.0


In [ ]:
df_normalized = df_nona.copy()

for column in df_normalized.columns:
    if column == 'date':
        continue
    df_normalized[column] = (df_normalized[column] - df_normalized[column].min()) / (df_normalized[column].max() - df_normalized[column].min()) * 2 + (-1)

df_normalized

,date,mintemp,maxtemp,meantemp,minpres,maxpres,meanpres,maxhumi,minhumi,meanhumi,meanwdspd,maxdewpt,mindewpt,meandewpt,N_wd,NW_wd,W_wd,SW_wd,S_wd,SE_wd,E_wd,NE_wd,nocld,little,some,much,cloudy,rain,mintemp_1,mintemp_2,mintemp_3,maxtemp_1,maxtemp_2,maxtemp_3,meantemp_1,meantemp_2,meantemp_3,minpres_1,minpres_2,minpres_3,maxpres_1,maxpres_2,maxpres_3,meanpres_1,meanpres_2,meanpres_3,maxhumi_1,maxhumi_2,maxhumi_3,minhumi_1,minhumi_2,minhumi_3,meanhumi_1,meanhumi_2,meanhumi_3,meanwdspd_1,meanwdspd_2,meanwdspd_3,maxdewpt_1,maxdewpt_2,maxdewpt_3,mindewpt_1,mindewpt_2,mindewpt_3,meandewpt_1,meandewpt_2,meandewpt_3,N_wd_1,N_wd_2,N_wd_3,NW_wd_1,NW_wd_2,NW_wd_3,W_wd_1,W_wd_2,W_wd_3,SW_wd_1,SW_wd_2,SW_wd_3,S_wd_1,S_wd_2,S_wd_3,SE_wd_1,SE_wd_2,SE_wd_3,E_wd_1,E_wd_2,E_wd_3,NE_wd_1,NE_wd_2,NE_wd_3,nocld_1,nocld_2,nocld_3,little_1,little_2,little_3,some_1,some_2,some_3,much_1,much_2,much_3,cloudy_1,cloudy_2,cloudy_3,rain_1,rain_2,rain_3
3,30.09.2012,0.215686,0.418182,0.331158,-0.132832,-0.079284,-0.077373,0.694118,-0.666667,-0.023393,-0.689574,0.345455,0.230769,0.283727,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.215686,0.490196,0.372549,0.381818,0.345455,0.236364,0.332894,0.421454,0.294692,-0.097744,-0.057644,-0.097744,-0.079284,-0.079284,-0.156010,-0.051531,-0.047687,-0.105564,0.576471,-0.364706,1.000000,-0.688889,-0.844444,-0.200000,-0.227885,-0.689853,0.481332,-0.625663,0.752992,-0.388279,0.345455,0.090909,0.563636,0.138462,-0.076923,0.323077,0.187761,-0.019471,0.481224,1.0,-1.0,1.0,1.0,1.0,-1.0,1.0,-1.0,-1.0,1.0,-1.0,-1.0,1.0,-1.0,-1.0,1.0,-1.0,-1.0,1.0,-1.0,-1.0,1.0,-1.0,-1.0,1.0,1.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
4,1.10.2012,-0.058824,0.345455,0.093263,-0.172932,0.079284,-0.032348,1.000000,-0.822222,0.232139,-0.670484,0.418182,-0.230769,0.210646,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.215686,0.215686,0.490196,0.418182,0.381818,0.345455,0.331158,0.332894,0.421454,-0.132832,-0.097744,-0.057644,-0.079284,-0.079284,-0.079284,-0.077373,-0.051531,-0.047687,0.694118,0.576471,-0.364706,-0.666667,-0.688889,-0.844444,-0.023393,-0.227885,-0.689853,-0.689574,-0.625663,0.752992,0.345455,0.345455,0.090909,0.230769,0.138462,-0.076923,0.283727,0.187761,-0.019471,1.0,1.0,-1.0,-1.0,1.0,1.0,-1.0,1.0,-1.0,-1.0,1.0,-1.0,-1.0,1.0,-1.0,-1.0,1.0,-1.0,-1.0,1.0,-1.0,-1.0,1.0,-1.0,1.0,1.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
5,2.10.2012,0.137255,0.381818,0.234495,-0.248120,-0.079284,-0.176897,1.000000,-0.533333,0.292022,-0.592186,0.527273,0.138462,0.357441,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-0.058824,0.215686,0.215686,0.345455,0.418182,0.381818,0.093263,0.331158,0.332894,-0.172932,-0.132832,-0.097744,0.079284,-0.079284,-0.079284,-0.032348,-0.077373,-0.051531,1.000000,0.694118,0.576471,-0.822222,-0.666667,-0.688889,0.232139,-0.023393,-0.227885,-0.670484,-0.689574,-0.625663,0.418182,0.345455,0.345455,-0.230769,0.230769,0.138462,0.210646,0.283727,0.187761,1.0,1.0,1.0,1.0,-1.0,1.0,1.0,-1.0,1.0,1.0,-1.0,1.0,1.0,-1.0,1.0,1.0,-1.0,1.0,1.0,-1.0,1.0,1.0,-1.0,1.0,1.0,1.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
6,3.10.2012,-0.019608,0.381818,0.214237,-0.248120,-0.038363,-0.167642,1.000000,-0.888889,0.027111,-0.513058,0.490909,-0.261538,0.165508,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.137255,-0.058824,0.215686,0.381818,0.345455,0.418182,0.234495,0.093263,0.331158,-0.248120,-0.172932,-0.132832,-0.079284,0.079284,-0.079284,-0.176897,-0.032348,-0.077373,1.000000,1.000000,0.694118,-0.533333,-0.822222,-0.666667,0.292022,0.232139,-0.023393,-0.592186,-0.670484,-0.689574,0.527273,0.418182,0.345455,0.138462,-0.230769,0.230769,0.357441,0.210646,0.283727,1.0,1.0,1.0,1.0,1.0,-1.0,1.0,1.0,-1.0,1.0,1.0,-1.0,1.0,1.0,-1.0,1.0,1.0,-1.0,1.0,1.0,-1.0,1.0,1.0,-1.0,1.0,1.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
7,4.10.2012,0

In [ ]:
df_normalized.corr()[['meantemp']].sort_values('meantemp').T

/tmp/ipykernel_219/2508009165.py:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  df_normalized.corr()[['meantemp']].sort_values('meantemp').T


,meanpres,meanpres_1,maxpres,maxpres_1,meanpres_2,maxpres_2,meanpres_3,maxpres_3,minpres,minpres_1,minpres_2,minpres_3,meanwdspd_1,meanwdspd_2,meanwdspd_3,meanwdspd,NW_wd,NW_wd_3,N_wd,N_wd_3,NW_wd_1,NW_wd_2,N_wd_2,N_wd_1,cloudy_2,cloudy_3,cloudy,cloudy_1,W_wd,W_wd_3,NE_wd,little_2,NE_wd_3,little_1,little,little_3,W_wd_2,W_wd_1,NE_wd_2,nocld_1,NE_wd_1,nocld_3,nocld,nocld_2,rain,some,rain_1,rain_2,much,SW_wd,rain_3,some_3,some_2,much_1,much_2,some_1,SW_wd_3,much_3,E_wd,SW_wd_2,E_wd_3,SW_wd_1,E_wd_2,E_wd_1,S_wd,S_wd_3,S_wd_2,S_wd_1,SE_wd,SE_wd_3,SE_wd_2,SE_wd_1,minhumi,minhumi_3,minhumi_2,maxhumi_3,minhumi_1,maxhumi_2,maxhumi,maxhumi_1,meanhumi_3,meanhumi,meanhumi_2,meanhumi_1,mindewpt_3,mindewpt_2,mindewpt,mindewpt_1,meandewpt_3,maxdewpt_3,maxdewpt_2,meandewpt_2,maxdewpt_1,meandewpt_1,meandewpt,maxdewpt,maxtemp_3,mintemp_3,maxtemp_2,mintemp_2,mintemp_1,maxtemp_1,meantemp_3,mintemp,meantemp_2,maxtemp,meantemp_1,meantemp
meantemp,-0.888291,-0.884047,-0.877442,-0.877294,-0.862123,-0.857301,-0.849329,-0.843851,-0.832205,-0.825793,-0.804286,-0.793083,-0.168622,-0.148217,-0.12873,-0.126531,-0.11651,-0.108752,-0.106826,-0.100569,-0.094822,-0.09394,-0.093232,-0.087293,-0.044946,-0.042317,-0.041403,-0.036049,-0.017963,-0.013365,-0.009491,-0.000559,0.000774,0.002434,0.003328,0.004337,0.006081,0.011069,0.014234,0.017199,0.019181,0.020419,0.024166,0.025272,0.028135,0.030129,0.031013,0.031659,0.033692,0.035432,0.035463,0.036888,0.038346,0.038692,0.038895,0.039471,0.042452,0.045934,0.053197,0.061853,0.069335,0.069406,0.083109,0.085941,0.117106,0.118317,0.135669,0.146929,0.151426,0.153418,0.166706,0.172652,0.287682,0.29533,0.303513,0.305239,0.312512,0.31455,0.326012,0.330528,0.335664,0.338495,0.349106,0.364659,0.826478,0.838962,0.851051,0.854844,0.879245,0.880397,0.889723,0.890736,0.906198,0.908301,0.911412,0.918413,0.92386,0.930355,0.934199,0.936123,0.948148,0.950347,0.958868,0.964997,0.966659,0.967442,0.981401,1.0
